# KE-T5: Korean-English T5 (한영번역)
* KE-T5는 Text-to-Text Transfer Transformer 모델을 한국어와 영어 코퍼스를 이용하여 사전학습한 모델
* 설명
  * https://github.com/AIRC-KETI/ke-t5
* huggingface
  * https://huggingface.co/KETI-AIR/ke-t5-large



## fine-tuning 방법과 PEFT QLoRA 방법 구현

### 필요한 패키지 설치 및 데이터셋 준비

In [1]:
# 필요 package 다운로드
!pip install -q wandb # 머신 러닝 실험을 추적하고 시각화
!pip install -q pynvml # NVIDIA GPU의 상태와 성능 모니터링에 사용
!pip install -q datasets==2.16.1 # Hugging Face의 데이터셋 라이브러리
!pip install -q transformers==4.36.2 # Hugging Face의 트랜스포머 기반 모델(예: BERT, GPT)을 위한 사전 훈련된 모델과 토크나이저를 포함
!pip install -q evaluate==0.4.1 # 모델 성능 평가 메트릭
!pip install -q bitsandbytes==0.42.0 # 8비트 최적화와 같은 고성능 컴퓨팅을 위한 도구를 제공
!pip install -q peft==0.7.1 # Parameter-efficient fine-tuning
!pip install -q accelerate==0.26.1 # Hugging Face에서 제공, 파이썬 스크립트를 GPU, TPU 또는 여러 GPU에 걸쳐 쉽게 실행할 수 있게 도와줌
!pip install -q tokenizer==3.4.3 # 언어와 데이터 형식을 처리
!pip install -q sentencepiece==0.1.99 # 텍스트 데이터를 서브워드 단위로 분할
!pip install -q sacrebleu # 번역의 품질을 평가

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.5/258.5 kB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 514.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

In [2]:
# gpu 확인
!nvidia-smi

Tue Feb 27 14:13:13 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [5]:
# 모든 데이터 사용 여부
USE_SMALL_DATASET = True
# 양자화 사용여부
USE_QUANTIZATION = True
# PEFT 사용 여부
USE_QLORA = True

In [4]:
# 특정 GPU 만 사용 하려면 설정 / 여러 GPU 사용 = "0, 1, 2"
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
# EN-KO dataset 설정
from datasets import load_dataset, Dataset, DatasetDict
import pandas as pd

# en_ko_dataset = load_dataset("bongsoo/news_talk_en_ko", column_names=["english", "korean"])

en_ko_dataset = load_dataset("bongsoo/news_talk_en_ko")
# 학습에 맞게 데이터셋 수정 (column을 english / korean 지정)
def fix_datasetform(dataset):
  dataset.set_format(type="pandas") # 판다스로 변환
  df = dataset["train"][:] # train 선택
  first_sample = df.columns.values

  df.columns = ("english", "korean")
  df = pd.concat([pd.DataFrame([first_sample], columns=df.columns), df]).reset_index(drop=True)

  dataset = Dataset.from_pandas(df)

  return DatasetDict({
      "train": dataset
  })

en_ko_dataset = fix_datasetform(en_ko_dataset)

In [6]:
# 데이터 확인
print(en_ko_dataset)
print(en_ko_dataset['train']['english'][:3])
print(en_ko_dataset['train']['korean'][:3])

DatasetDict({
    train: Dataset({
        features: ['english', 'korean'],
        num_rows: 1300000
    })
})
["Skinner's reward is mostly eye-watering.", 'Even some problems can be predicted.', 'Only God will exactly know why.']
['스키너가 말한 보상은 대부분 눈으로 볼 수 있는 현물이다.', '심지어 어떤 문제가 발생할 건지도 어느 정도 예측이 가능하다.', '오직 하나님만이 그 이유를 제대로 알 수 있을 겁니다.']


### train / valid / test 분할
* train : test = 0.8 : 0.2
* valid 는 test set 에서 5:5




In [6]:
train_test_dataset = en_ko_dataset['train'].train_test_split(test_size=0.2)
valid_test_dataset = train_test_dataset['test'].train_test_split(test_size=0.5)

train_dataset = train_test_dataset['train']
valid_dataset = valid_test_dataset['train']
test_dataset = valid_test_dataset['test']

# 빠르게 부분 학습만 / 실제 학습시는 False 로 수정
seed = 1
if USE_SMALL_DATASET:
  train_dataset = train_dataset.shuffle(seed=seed).select(range(1000))
  valid_dataset = valid_dataset.shuffle(seed=seed).select(range(100))
  test_dataset = test_dataset.shuffle(seed=seed).select(range(100))


en_ko_datasets = DatasetDict({
    "train": train_dataset,
    "validation": valid_dataset,
    "test": test_dataset
})

In [8]:
en_ko_datasets

DatasetDict({
    train: Dataset({
        features: ['english', 'korean'],
        num_rows: 1000
    })
    validation: Dataset({
        features: ['english', 'korean'],
        num_rows: 100
    })
    test: Dataset({
        features: ['english', 'korean'],
        num_rows: 100
    })
})

## Tokenizer

In [9]:
MODEL_ID = 'KETI-AIR/ke-t5-large-ko'
import transformers
from transformers import AutoTokenizer
# use_fast=True 는 Rust로 구축된 Tokenizer로 속도를 빠르게 해줌 (단, 지원시만 사용가능)
# 호완성 체크를 위해 false 로 선언 후 아래와 같이 fast 지원 버전인지 확인 후 사용
# tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, use_fast=False)
# isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, use_fast=True)

tokenizer_config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/604 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.47M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [10]:
# Tokenizer vocab 순서대로 확인
# 특정 객체의 사용가능한 메서드와 속성 확인
# print(dir(tokenizer))
# print(help(tokenizer))

sorted_vocab = sorted(tokenizer.get_vocab().items(), key=lambda item: item[1])
sorted_tokens = [{index, token} for token, index in sorted_vocab if index < 10]
sorted_tokens

[{0, '<pad>'},
 {1, '</s>'},
 {2, '<unk>'},
 {'.', 3},
 {',', 4},
 {5, '▁the'},
 {6, '의'},
 {7, '▁'},
 {8, 's'},
 {9, '에'}]

In [11]:
# encoder 와 decoder 의 tokenizer 확인 => 이번의 경우 tokenizer가 encoder / decoder 동일하게 사용하는 케이스
# encoder 와 decoder 의 tokenizer 가 서로 다른 경우가 존재 (생각: 각각 존재하는 경우 선택 폭이 좁아져 정확해지거나 판단이 빨라지지 않을까?)
en0 = en_ko_datasets['train']['english'][0]
ko0 = en_ko_datasets['train']['korean'][0]
en0_tokenizer = tokenizer(en0)
# target tokenizer 로 tokenize
ko0_tokenizer = tokenizer(text_target = ko0)

print(f'tokenizer 결과 확인')
# input_ids: 입력 텍스트를 토크나이저가 처리한 후 얻어진 토큰의 ID
# attention_mask: 실제 데이터와 패딩(padding)을 구분 (데이터: 1 / 패딩: 0)
# labels: 보통 시퀀스-투-시퀀스 작업에서 모델이 예측해야 할 출력 ex) 원본 텍스트는 input_ids로, 번역된 텍스트는 labels로 표현
# labels 는 아래와 같은 경우에 생성 : tokenizer(inputs, text_target, max_length, truncation=True)
print(f'keys: {en0_tokenizer.keys()}')
print(f'en0: {en0}')
print(f'ko0: {ko0}')

print(f'en0_ids: {en0_tokenizer["input_ids"]}')
print(f'ko0_ids: {ko0_tokenizer["input_ids"]}')
print(f'en0_decode: {tokenizer.decode(en0_tokenizer["input_ids"])}')
print(f'ko0_decode: {tokenizer.decode(ko0_tokenizer["input_ids"])}')

tokenizer 결과 확인
keys: dict_keys(['input_ids', 'attention_mask'])
en0: Maybe it was a video he witnessed at a science museum in Spain.
ko0: 어쩌면 그건 그가 스페인의 한 과학관에서 목격한 영상이었을지 모른다.
en0_ids: [14595, 60, 62, 16, 1989, 92, 31179, 65, 16, 5487, 13986, 20, 15792, 3, 1]
ko0_ids: [11368, 6860, 1435, 3934, 6, 59, 3478, 15384, 39577, 2034, 32802, 76, 5249, 3, 1]
en0_decode: Maybe it was a video he witnessed at a science museum in Spain.</s>
ko0_decode: 어쩌면 그건 그가 스페인의 한 과학관에서 목격한 영상이었을지 모른다.</s>


## prefix 생성

In [12]:
max_token_length = 64 # 토크나이저가 생성할 토큰 시퀀스의 최대 길이
source_lang = 'english'
target_lang = 'korean'
prefix = f'translate {source_lang} to {target_lang}:'

def add_prefix(lang):
  inputs = [prefix + source for source in lang[source_lang]]
  targets = [target for target in lang[target_lang]]
  # truncation: max_length보다 긴 입력이 들어올 경우, 그 입력 자름
  return tokenizer(inputs, text_target=targets, max_length=max_token_length, truncation=True)


In [13]:
# 전체 데이터 Tokenize
tokenized_datasets = en_ko_datasets.map(
                        add_prefix,
                        batched=True,
                        remove_columns=en_ko_datasets["train"].column_names,
                     )


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [14]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1000
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 100
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 100
    })
})

## Model

In [15]:
from transformers import AutoModelForSeq2SeqLM
from transformers import BitsAndBytesConfig
import torch
# 가중치와 활성화를 8비트 정수(int8)로 변환
if USE_QUANTIZATION:
  bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, # 4비트로 데이터를 로드 / 데이터를 저장하거나 처리할 때 메모리 사용량을 줄이기 위해 사용
    bnb_4bit_use_double_quant=True, # 이중 양자화를 사용 / 성능 손실을 최소화하면서 메모리 사용량을 줄임
    bnb_4bit_quant_type="nf4", # 양자화 유형을 "nf4"로 설정
    bnb_4bit_compute_dtype=torch.bfloat16 # 계산에 사용되는 데이터 타입 (16비트 부동 소수점 형식) / fp32에 비해 bfloat16 은 메모리 사용량이적음(하지만 낮은 정밀도)
  )

  model = AutoModelForSeq2SeqLM.from_pretrained(
      MODEL_ID,
      device_map="auto", # 모델의 각 부분을 여러 GPU에 분산시켜, 메모리 부족 문제를 방지하고 계산 효율성을 높임 => 현재 모델은 안되는듯
      quantization_config=bnb_config,
      trust_remote_code=True # 사용자 정의 코드를 믿고 로드 => 보안 위험이있으나 커스텀된 모델 사용시 유용
  )

  # 이미 양자화를 하여 아래는 중복으로 양자화라 수행 X
  # 또한 바로 model 을 적용하기엔 아래 오류나며 수행되지않음(모델이나 텐서가 올바르게 구성되지 않았을 때 발생)
  # RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn
  # 아래 코드 실행
  # 주어진 모델의 구조를 분석하고, 모델의 가중치와 활성화를 k비트 정밀도로 표현할 수 있도록 변환
  # k비트 형식으로 양자화되며, 훈련 중에는 이 양자화된 가중치를 사용하여 연산이 수행
  from peft import prepare_model_for_kbit_training
  model = prepare_model_for_kbit_training(model)

  # peft 신경망 모델의 정밀도를 낮추어 인퍼런스 속도를 향상시키는 데 도움을 주는 Python 라이브러리
  # 모델을 INT8 (8-bit 정수) 형식으로 변환하여 모델의 메모리 요구 사항을 줄이고 추론 속도를 높임
  # from peft import prepare_model_for_int8_training
  # model = prepare_model_for_int8_training(model)


else:
  model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_ID)

pytorch_model.bin:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

In [16]:
from transformers import DataCollatorForSeq2Seq
# 모델의 훈련 과정에서 배치(batch) 데이터를 자동으로 모델에 적합한 형태로 변환
# 모든 시퀀스를 동일한 길이로 맞추기 위해 자동으로 패딩(padding)을 추가
# 실제 길이와 패딩된 부분을 구분하는 Attention Mask 생성
# 대상(target) 시퀀스 Label 도 적절히 Padding을 추가 처리 (decoder_input_ids는 앞에 0 즉, Padding symbol 이 추가) => 하나씩 오른쪽으로 shift
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
# # 아래와 같이도 가능
# from transformers import DataCollatorForLanguageModeling
# # mlm=False 는 Masked Language Modeling (MLM)"을 사용하지 않겠다는 의미
# # 즉, 마스킹된 언어 모델링 대신에 전통적인 순차적 언어 모델링(Causal Language Modeling, CLM)을 사용하겠다는 것을 의미
# data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)


### DataCollatorForSeq2Seq 확인


In [17]:
sample_data = [tokenized_datasets['train'][i] for i in range(0, 2)]
print('before collator')
print(f'keys:  {sample_data[0].keys()}')
print(f'intput_ids: {sample_data[0]["input_ids"]}')
print(f'attention_mask:  {sample_data[0]["attention_mask"]}')
print(f'labels: {sample_data[0]["labels"]}')
print('-' * 200)

# list 로 들어가야하는듯
collator_data = data_collator(sample_data)
print('after collator')
print(f'keys:  {collator_data.keys()}') # decoder_input_ids 생김
print(f'intput_ids: {collator_data["input_ids"]}') # tensor 로 변함, 뒤에 0 으로 길이 맞춰줌
print(f'attention_mask:  {collator_data["attention_mask"]}') # 실데이터와 맞춰주기 위해 들어간 pad 확인
print(f'labels: {collator_data["labels"]}') # tensor 로 변함, -100 으로 길이 맞춰줌
print(f'decoder_input_ids: {collator_data["decoder_input_ids"]}') # 앞에 0 으로 pad symbol 이 추가

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


before collator
keys:  dict_keys(['input_ids', 'attention_mask', 'labels'])
intput_ids: [35017, 7, 16825, 26854, 10, 7, 39508, 477, 103, 47751, 2403, 60, 62, 16, 1989, 92, 31179, 65, 16, 5487, 13986, 20, 15792, 3, 1]
attention_mask:  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
labels: [11368, 6860, 1435, 3934, 6, 59, 3478, 15384, 39577, 2034, 32802, 76, 5249, 3, 1]
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
after collator
keys:  dict_keys(['input_ids', 'attention_mask', 'labels', 'decoder_input_ids'])
intput_ids: tensor([[35017,     7, 16825, 26854,    10,     7, 39508,   477,   103, 47751,
          2403,    60,    62,    16,  1989,    92, 31179,    65,    16,  5487,
         13986,    20, 15792,     3,     1],
        [35017,     7, 16825, 26854,    10,     7, 39508,   477,   103, 14438,
         

## Inference Test (학습전 상태 추론 확인)

In [18]:
# 데이터 배치를 n번째 GPU로 이동 / 빠른 속도로 연산을 수행 (모델과 데이터가 동일 장치에 있을때 효율적)
# collator_data.to('cuda:0')

# 미리 정의된 신경망 모델에 변수를 입력으로 넣고, 그 결과를 'outputs'에 저장
# ** 은 argument unpacking => 딕셔너리 내의 모든 요소를 키워드 인자로 모델에 전달
# 입력 데이터에 대한 예측 또는 결과를 outputs라는 변수에 저장
outputs = model(**collator_data)
# OrderedDict (추가된 순서를 기억)
# loss : 모델의 예측과 실제 값 사이의 차이 => loss 가 낮을수록 모델의 예측이 실제 데이터에 더 가까움
# logits: 모델이 출력한 로짓(logit) 값, 마지막 선형 레이어에서 나온 원시 출력 값 => 예측확률
# past_key_values: 이전 단계의 키(key)와 값(value) 쌍을 포함 => 이전 단어의 정보를 저장하여 다음 단어를 예측할 때 사용
# encoder_last_hidden_state: 인코더의 마지막 은닉 상태
outputs.keys()

odict_keys(['loss', 'logits', 'past_key_values', 'encoder_last_hidden_state'])

In [19]:
import torch
# argmax 가장 큰 값을 가지는 인덱스를 반환
# n 번째 데이터에 대해 가장 확신하는 클래스의 인덱스를 나타내는 NumPy 배열
best_output = torch.argmax(outputs['logits'][0], axis=1).cpu().numpy()
print(f'input: {tokenizer.convert_ids_to_tokens(collator_data["input_ids"][0])}')
print(f'best_output: {tokenizer.convert_ids_to_tokens(best_output)}')

input: ['▁translate', '▁', 'eng', 'lish', '▁to', '▁', 'korea', 'n', ':', 'May', 'be', '▁it', '▁was', '▁a', '▁video', '▁he', '▁witnessed', '▁at', '▁a', '▁science', '▁museum', '▁in', '▁Spain', '.', '</s>']
best_output: ['▁Gill', '▁어쩌면', 'iq', '▁전격적으로', 'iq', '정혁', 'iq', '영장실질심사', '▁fit', 'ai', '정혁', 'iq', '딧', '딧', '딧']


### Logging Tool 설정
* wandb: 모델 트레이닝 과정의 모니터링, 시각화 및 관리

In [20]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

### evlauation Metric 생성
* sacreBLEU : 일반적인 BLEU 계산과 달리 sacreBLEU는 일관된 데이터셋과 토크나이징 규칙을 사용하여 BLEU 점수를 보다 신뢰성 있게 만들어줌
* BLEU (BiLingual Evaluation Understudy)
  * 기계 번역의 품질을 평가하는 데 사용되는 메트릭으로, 모델이 생성한 번역문이 얼마나 참조 번역문(reference translation)과 유사한지를 측정
  * BLEU 점수는 0부터 100 사이의 값으로 나타나며, 높은 점수는 모델의 번역이 참조 번역과 더 유사함을 의미

In [21]:
from datasets import load_metric
import numpy as np

metric = load_metric("sacrebleu")
def compute_metrics(eval_preds):
    preds = eval_preds.predictions
    labels = eval_preds.label_ids

    if isinstance(preds, tuple):
        preds = preds[0]
    # 특수 토큰은 디코딩 과정에서 제외
    # ex: 시작 토큰(<s>), 종료 토큰(</s>), 패딩 토큰(<pad>), 미지의 단어 표시 토큰(<unk>)
    decoded_preds = tokenizer.batch_decode(preds[0], skip_special_tokens=True)

    # Replace -100s in the labels as we can't decode them
    # np.where(조건, 참일 때 값, 거짓일 때 값)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": result["score"]}

<ipython-input-21-8d2b17f1fd7a>:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("sacrebleu")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for sacrebleu contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/sacrebleu/sacrebleu.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


## PEFT Fine-Tuning
* 아래 lora_config 설정시 PEFT Fine-Tuning 수행

In [22]:
# gradient checkpointing: 메모리 효율을 높이기 위한 기법
# 모델은 훈련 중에 중간 계산 결과(즉, 체크포인트)를 저장하여, 그라디언트를 계산할 때 다시 사용할 수 있음
# 즉, 모든 중간 계산 결과를 메모리에 저장하는 대신 필요할 때만 다시 계산하여 메모리 사용량을 줄일 수 있음
# 단, 이 방법은 계산 시간을 증가시킬수 있음
model.gradient_checkpointing_enable()

### QLoRA( Quantized LLMs with Low-Rank Adapters)

In [23]:
from peft import LoraConfig, TaskType, get_peft_model
# 쿼리(query)와 키(key), 값(value), 출력(output)
# PEFT 라이브러리를 사용하면 대부분의 원래 모델 가중치를 고정하고 훨씬 작은 추가 파라미터 세트를 훈련하여 모델 레이어를 교체하거나 확장
if USE_QLORA:
  lora_config = LoraConfig(
      task_type=TaskType.SEQ_2_SEQ_LM, # 작업 유형 ex CAUSAL_LM (주어진 텍스트의 다음 부분을 예측하는 작업)
      r=8, # LoRA에서 사용되는 저랭크 행렬의 랭크, 즉 저랭크 행렬의 차원
      lora_alpha=32, # LoRA 어댑터의 확장 계수 / 어댑터가 입력에 미치는 영향의 정도를 결정 즉, 더 높은 값은 어댑터가 더 강한 영향을 미침
      lora_dropout=0.1, # 모델의 과적합을 방지하기 위해 일부 뉴런을 무작위로 비활성화하는 기법
      target_modules=["q" ,"v"], # 특정 레이어나 기능에 LoRA를 적용
      bias="none", # none: 편향을 사용하지 않음 => 편향을 추가하면 모델의 표현력을 높일 수 있지만 모델의 복잡성을 증가
  )

  model = get_peft_model(model, lora_config)


### Train

In [24]:
# 트레이닝 결과 저장할 폴더 생성
import os
folder_path ='./model_output'
log_path = './logs'
result_path = './results'

if not os.path.exists(folder_path):
    os.makedirs(folder_path)
    print(f"Folder '{folder_path}' created.")
else:
    print(f"Folder '{folder_path}' already exists.")

if not os.path.exists(log_path):
    os.makedirs(log_path)
    print(f"Folder '{log_path}' created.")
else:
    print(f"Folder '{log_path}' already exists.")

if not os.path.exists(result_path):
    os.makedirs(result_path)
    print(f"Folder '{result_path}' created.")
else:
    print(f"Folder '{result_path}' already exists.")

Folder './model_output' created.
Folder './logs' created.
Folder './results' created.


In [25]:
# 모델 args 설정
from transformers import Seq2SeqTrainingArguments
training_args = Seq2SeqTrainingArguments(
    f"{MODEL_ID}-finetuned",     # 모델과 체크포인트 저장 경로
    per_device_train_batch_size=8,   # 각 디바이스당 훈련 배치 크기 => 각 GPU/CPU에서 사용할 훈련 배치 크기
    per_device_eval_batch_size=8,    # 각 디바이스당 평가 배치 크기
    auto_find_batch_size=True,      # 시스템의 메모리 용량과 계산 능력을 고려하여 최적의 배치 크기를 자동으로 결정
    num_train_epochs=2,             # 훈련할 총 에포크 수
    learning_rate=5e-5,             # 학습률
    warmup_steps=10,               # 트레이닝 초기에 학습률을 서서히 증가시키는 단계의 수를 지정 => 배치 처리 반복 횟수에 대한 상대적인 값 batch_size * n
    weight_decay=0.01,              # 가중치 감소
    save_total_limit=2,             # 저장할 총 체크포인트 수 제한
    save_strategy="epoch",    # 평가 전략 ("epoch"는 각 에포크 끝에서 평가) => 모델을 얼마나 자주 평가할지 설정
    report_to="wandb",              # Weights & Biases (W&B) 로깅
    logging_dir='./logs',           # 로그 저장 경로
    logging_steps=2,              # 로깅을 위한 스텝 간격
    do_train=True,                  # 훈련 수행 여부
    do_eval=True,                    # 평가 수행 여부
    # bf16=True                       # 훈련 과정에서 bfloat16 즉, 16비트 부동소수점 사용 => 단 사용하려면  torch>=1.10, using Ampere GPU with cuda>=11.0 필요
)

In [26]:
# train 설정
from transformers import Seq2SeqTrainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [27]:
# 모델 트레이닝
# warnings 제거
# import warnings
# warnings.filterwarnings("ignore", category=UserWarning)

trainer.train()

wandb: Currently logged in as: wotres. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Step,Training Loss
2,17.823400
4,18.411900
6,17.762700
8,17.792800
10,17.342700
12,18.716600
14,17.587000
16,17.530500
18,18.073000
20,17.680500


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


TrainOutput(global_step=250, training_loss=14.371252883911133, metrics={'train_runtime': 261.3759, 'train_samples_per_second': 7.652, 'train_steps_per_second': 0.956, 'total_flos': 508345714114560.0, 'train_loss': 14.371252883911133, 'epoch': 2.0})

In [28]:
# 최종 모델 저장
trainer.save_model("./results")
wandb.finish()

train/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
train/learning_rate,▂▇███▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁
train/loss,█████▆▇▅▆▆▅▄▅▃▄▆▆▃▃▄▃▃▂▂▃▁▂▄▂▂▁▂▂▁▂▁▃▃▁▅
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁
train/train_samples_per_second,▁
train/train_steps_per_second,▁
train/epoch,2.0
train/global_step,250


## Inference (Trained Model with PEFT)

In [1]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, BitsAndBytesConfig
import torch
# 모델 로드
model_dir = "./results"
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForSeq2SeqLM.from_pretrained(model_dir)
# bnb_config = BitsAndBytesConfig(
#   load_in_4bit=True,
#   bnb_4bit_use_double_quant=True,
#   bnb_4bit_quant_type="nf4",
#   bnb_4bit_compute_dtype=torch.bfloat16
# )
# model = AutoModelForSeq2SeqLM.from_pretrained(model_dir, quantization_config=bnb_config)
model


# 모델의 모든 파라미터와 버퍼를 CPU 메모리로 이동
# gpu 로 옮기는건 model.to(device)
model.cpu()
# 위 코드는 아래 에러 발생 (모델이 GPU 상에서 실행되고 있고, 입력 데이터 또는 모델이 CPU에 있거나 그 반대의 경우 발생)
# RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0!

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


T5ForConditionalGeneration(
  (shared): Embedding(64128, 1024)
  (encoder): T5Stack(
    (embed_tokens): Embedding(64128, 1024)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): lora.Linear(
                (base_layer): Linear(in_features=1024, out_features=1024, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=1024, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=1024, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k): Linear(in_features=1024, out_features=1024, bias=False)
         

In [8]:
prefix = f'translate english to korean:'
input_text = [prefix + en for en in en_ko_datasets['test']['english'][:2]]
inputs = tokenizer(input_text,
                   return_tensors="pt",
                   padding=True,
                   max_length=64)

print(f'inputs[0]: {tokenizer.decode(inputs["input_ids"][0])}')
print('테스트 될 tokenizer된 inputs 형식')
inputs

inputs[0]: translate english to korean:I think the hospitality of Busan in 2002 Asian Games remains in my memory.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
테스트 될 tokenizer된 inputs 형식


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2636: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


{'input_ids': tensor([[35017,     7, 16825, 26854,    10,     7, 39508,   477,   103,   359,
           721,     5, 53470,    14, 30602,  1284,    20,  3338, 11472, 14553,
          4423,    20,   451, 10199,     3,     1,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
        [35017,     7, 16825, 26854,    10,     7, 39508,   477,   103, 21248,
            38,   248,     4,     5,  1655,   720,  2417,    81,  3278,  8571,
           384,  5097, 38221,  6285,     5,   765, 17315,    54,   414,  1810,
          2403,  7333, 34755,     4,   134,  2475,     5,   765,    44,  4508,
             4,  4519,   175,   505,  1541,    20,  5383,   248,     3,     1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 

In [10]:
# output 추론
"""
max_length vs max_new_tokens
max_length 는 입력 프롬프트의 길이와 max_new_tokens의 합 즉, Prompt(초기 입력값)을 포함한 최대 길이를 지정
max_new_tokens 는 Prompt를 제외한 생성된 문장의 최대 길이를 지정
max_length 보다 max_new_tokens 이 우선적 적용됨
참고만,
Causal Language Models (GPT 같은 모델)에서는 다음 토큰을 순차적으로 생성하니 max_new_tokens을 사용하고
T5와 같은 다목적 LM에서는 전체 출력의 길이를 제한을 위해 max_length를 사용하는 경우가 있다고 함
"""
outputs = model.generate(
    **inputs,
    max_length=64, # 생성된 텍스트의 전체 길이 토큰 수 제한
    num_beams=5, # 빔 검색은 모델이 다음에 생성할 단어를 선택할 때, 여러 가지 후보를 고려하고 그 중 가장 가능성 있는 후보를 선택
    # max_new_tokens= 24, # 새로 생성되는 토큰 수
)


In [11]:
outputs_text = [tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(output)) for output in outputs]
# 아래 동일
# outputs_text = [tokenizer.decode(output) for output in outputs]

outputs_text

['<pad> Gill 보성군 보성군borborborborborborborborborborborborborborborborborborborborborborborborborborborborborborborborborborborborborborbor말씀말씀말씀말씀말씀말씀말씀말씀혜린혜린말씀말씀말씀말씀말씀말씀말씀말씀말씀말씀말씀',
 '<pad> Gill Study Study Study Study Study Study Study Study Study Study Study Study Study Study Study Study Study Study Study Study Study Study Study 해결사 해결사 해결사 해결사 해결사 해결사 해결사 해결사 해결사 해결사 해결사 해결사 해결사 해결사 해결사 해결사 해결사 해결사 해결사 해결사 해결사 해결사 해결사 해결사 해결사 해결사 해결사 해결사 해결사 해결사 해결사 해결사 해결사 해결사 해결사 해결사 해결사 해결사 해결사']

In [12]:
outputs.shape

torch.Size([2, 64])